# Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
from matplotlib import colors as mcolors
from mycolorpy import colorlist as mcp
import matplotlib
import geopandas as gpd
from datetime import datetime, timedelta
import jenkspy
import warnings
from statsmodels.tsa.seasonal import seasonal_decompose
warnings.filterwarnings("ignore")


# Define country and parameters

In [2]:
# Select target country
country = 'Colombia'

# Set country-specific parameters: ISO codes and buffer size (in meters)
if country == 'Argentina':
    country_short = 'ARG'   # ISO 3-letter code
    country_code = 'AR'     # ISO 2-letter code
elif country == 'Chile':
    country_short = 'CHL'
    country_code = 'CL'
elif country == 'Colombia':
    country_short = 'COL'
    country_code = 'CO'
# Uncomment the following if Mexico is to be included in the analysis
# elif country == 'Mexico':
#     country_short = 'MEX'
#     country_code = 'MX'

# Set working directory

In [3]:
# Define working directory path
wd = (
#     '/your/path/to/working/directory/'
    '/Users/carmen/Library/CloudStorage/OneDrive-TheUniversityofLiverpool/research/latin-mobility-covid'
)

# Set parameters: place of interest? and stringency as predictor? 
And import necessary data

In [ ]:
place = 'capital' # 'fuas' or 'capital'
stringency = False # or False

if stringency == True:
    df = pd.read_csv(wd + '/data/outputs/' + country_short + '/mov-analysis/by-' + category + '/data_trend_netflows_' + place + '_param_tidy_results_cat.csv')
    df_m3_re = pd.read_csv(wd + '/data/outputs/' + country_short + '/mov-analysis/by-' + category + '/data_trend_param_' + place + '_m3cat_re.csv')
    df_m4_re = pd.read_csv(wd + '/data/outputs/' + country_short + '/mov-analysis/by-' + category + '/data_trend_param_' + place + '_m4cat_re.csv')
    df_m5_re = pd.read_csv(wd + '/data/outputs/' + country_short + '/mov-analysis/by-' + category + '/data_trend_param_' + place + '_m5cat_re.csv')
    df_m6_re = pd.read_csv(wd + '/data/outputs/' + country_short + '/mov-analysis/by-' + category + '/data_trend_param_' + place + '_m6cat_re.csv')
    df_m7_re = pd.read_csv(wd + '/data/outputs/' + country_short + '/mov-analysis/by-' + category + '/data_trend_param_' + place + '_m7cat_re.csv')
else:
    df = pd.read_csv(wd + '/data/outputs/' + country_short + '/mov-analysis/by-' + category + '/data_trend_netflows_' + place + '_param_tidy_results_cats.csv')
    df_m3_re = pd.read_csv(wd + '/data/outputs/' + country_short + '/mov-analysis/by-' + category + '/data_trend_param_' + place + '_m3cats_re.csv')
    df_m4_re = pd.read_csv(wd + '/data/outputs/' + country_short + '/mov-analysis/by-' + category + '/data_trend_param_' + place + '_m4cats_re.csv')
    df_m5_re = pd.read_csv(wd + '/data/outputs/' + country_short + '/mov-analysis/by-' + category + '/data_trend_param_' + place + '_m5cats_re.csv')
    df_m6_re = pd.read_csv(wd + '/data/outputs/' + country_short + '/mov-analysis/by-' + category + '/data_trend_param_' + place + '_m6cats_re.csv')
    df_m7_re = pd.read_csv(wd + '/data/outputs/' + country_short + '/mov-analysis/by-' + category + '/data_trend_param_' + place + '_m7cats_re.csv')

In [ ]:
# Select model
model = 6

# Filter fixed effects by selected model
df_model = df[df['model'] == model].reset_index(drop=True)

# Extract fixed effect intercept
fe_intercept = df_model[df_model['term'].str.contains('Intercept')]['estimate'].reset_index(drop=True).loc[0]

# Compute total intercepts: fixed + random effects
if model == 6:
    re_intercept = df_m6_re[df_m6_re['facet'].str.contains('Intercept')].reset_index(drop=True)
    intercept = [fe_intercept + re_intercept['estimate'].loc[i] for i in range(len(np.unique(df_m6_re['term'])))]
elif model == 5:
    re_intercept = df_m5_re[df_m5_re['facet'].str.contains('Intercept')].reset_index(drop=True)
    intercept = [fe_intercept + re_intercept['estimate'].loc[i] for i in range(len(np.unique(df_m5_re['term'])))]

# Extract fixed effect slope
fe_slope = df_model[df_model['term'] == 'time']['estimate'].reset_index(drop=True).loc[0]

# Compute total slopes: fixed + random effects
if model == 6:
    re_slope = df_m6_re[df_m6_re['facet'] == 'time'].reset_index(drop=True)
    slope = [fe_slope + re_slope['estimate'].loc[i] for i in range(len(np.unique(df_m6_re['term'])))]
elif model == 5:
    re_slope = df_m5_re[df_m5_re['facet'] == 'time'].reset_index(drop=True)
    slope = [fe_slope + re_slope['estimate'].loc[i] for i in range(len(np.unique(df_m5_re['term'])))]

# Handle stringency interaction term if present
try:
    stringency = [
        df_model[df_model['term'] == 'stringency_index']['estimate'].reset_index(drop=True).loc[0]
        for i in range(len(intercept))
    ]
except:
    stringency = [0 for i in range(len(intercept))]

# Add stringency effect to slopes
slope = [slope[i] + stringency[i] for i in range(len(slope))]

# Extract quadratic term for time if model == 6
if model == 6:
    quad = [
        df_model[df_model['term'] == 'time2']['estimate'].reset_index(drop=True).loc[0]
        for i in range(len(intercept))
    ]

# --- Standard errors ---

# Intercept standard error
fe_intercept_std = df_model[df_model['term'].str.contains('Intercept')]['std.error'].reset_index(drop=True).loc[0]

# Random intercept confidence intervals
if model == 6:
    re_intercept = df_m6_re[df_m6_re['facet'].str.contains('Intercept')].reset_index(drop=True)
elif model == 5:
    re_intercept = df_m5_re[df_m5_re['facet'].str.contains('Intercept')].reset_index(drop=True)

re_intercept_ci = [
    re_intercept.loc[i, 'conf.high'] - re_intercept.loc[i, 'conf.low']
    for i in range(len(re_intercept))
]

# Convert CI to standard deviation assuming normal distribution
re_intercept_std = [ci / (2 * 1.96) for ci in re_intercept_ci]

# Combine fixed + random intercept std if desired (currently only fixed used)
# intercept_std = [np.sqrt(fe_intercept_std**2 + std**2) for std in re_intercept_std]
intercept_std = [np.sqrt(fe_intercept_std**2) for std in re_intercept_std]

# Time slope standard error
fe_time_std = df_model[df_model['term'] == 'time']['std.error'].reset_index(drop=True).loc[0]

# Stringency std error (if present)
try:
    fe_stringency_std = df_model[df_model['term'] == 'stringency_index']['std.error'].reset_index(drop=True).loc[0]
except:
    fe_stringency_std = 0

# Random slope confidence intervals
if model == 6:
    re_time = df_m6_re[df_m6_re['facet'].str.contains('time')].reset_index(drop=True)
elif model == 5:
    re_time = df_m5_re[df_m5_re['facet'].str.contains('time')].reset_index(drop=True)

re_time_ci = [
    re_time.loc[i, 'conf.high'] - re_time.loc[i, 'conf.low']
    for i in range(len(re_time))
]

# Convert CI to standard deviation assuming normal distribution
re_time_std = [ci / (2 * 1.96) for ci in re_time_ci]

# Combine fixed stds only (random component excluded in current calc)
# slope_std = [np.sqrt(fe_time_std**2 + fe_stringency_std**2 + std**2) for std in re_time_std]
slope_std = [np.sqrt(fe_time_std**2 + fe_stringency_std**2) for std in re_time_std]

# Quadratic term std error
if model == 6:
    fe_quad_std = df_model[df_model['term'] == 'time2']['std.error'].reset_index(drop=True).loc[0]
if model == 5:
    fe_quad_std = 0

quad_std = [fe_quad_std for i in range(len(slope_std))]

# Compute recovery time for recovery thresholds 25, 50, 75, 100

In [ ]:
thresholds = [25, 50, 75, 100]  # Percentage thresholds to evaluate
times = []       # Store calculated times for each threshold
times_std = []   # Store standard deviations of times for each threshold

for thr in thresholds:
    times_thr = []      # Times for the current threshold across groups
    times_thr_std = []  # Std devs for the current threshold across groups
    
    for i in range(len(intercept)):
        # If both intercept and slope are positive, time threshold undefined (set -1)
        if intercept[i] > 0 and slope[i] > 0:
            times_thr.append(-1)
            times_thr_std.append(-1)
        
        else:  
            # Model 6: quadratic form: solve quadratic equation for time_thr
            if model == 6:
                discriminant = slope[i]**2 - 4 * quad[i] * (thr / 100) * intercept[i]
                
                # Calculate roots only if discriminant is non-negative
                if discriminant >= 0:
                    # Calculate first root
                    time_thr = (-slope[i] + np.sqrt(discriminant)) / (2 * quad[i])
                    
                    if time_thr > 0:
                        times_thr.append(time_thr)
                        # Partial derivatives for error propagation
                        partial_intercept = (-(thr / 100)) / np.sqrt(discriminant)
                        partial_slope = (1 / quad[i]) * (-1 + slope[i] / np.sqrt(discriminant))
                        partial_quad = (-(thr / 100) * intercept[i]) / (quad[i] * np.sqrt(discriminant)) + \
                                       (slope[i] - np.sqrt(discriminant)) / (2 * quad[i]**2)
                    
                    else:
                        # Calculate second root if first root is negative
                        time_thr = (-slope[i] - np.sqrt(discriminant)) / (2 * quad[i])
                        times_thr.append(time_thr)
                        # Partial derivatives for error propagation (alternate root)
                        partial_intercept = (thr / 100) / np.sqrt(discriminant)
                        partial_slope = (1 / quad[i]) * (-1 - slope[i] / np.sqrt(discriminant))
                        partial_quad = ((thr / 100) * intercept[i]) / (quad[i] * np.sqrt(discriminant)) + \
                                       (slope[i] + np.sqrt(discriminant)) / (2 * quad[i]**2)
                    
                    # Calculate standard deviation for time_thr via error propagation
                    time_thr_std = np.sqrt(
                        (partial_intercept * intercept_std[i])**2 +
                        (partial_slope * slope_std[i])**2 +
                        (partial_quad * quad_std[i])**2
                    )
                    times_thr_std.append(time_thr_std)
                
                else:
                    # If discriminant is negative, solution is complex -> NaN
                    times_thr.append(np.nan)
                    times_thr_std.append(np.nan)
            
            # Model 5: linear form, direct calculation of time_thr
            elif model == 5:
                time_thr = - (thr / 100) * (intercept[i] / slope[i])
                times_thr.append(time_thr) 
                
                # Partial derivatives for error propagation in linear model
                partial_intercept = -(thr / 100) / slope[i]
                partial_slope = ((thr / 100) * intercept[i]) / (slope[i]**2)
                
                # Calculate std deviation for time_thr
                time_thr_std = np.sqrt(
                    (partial_intercept * intercept_std[i])**2 +
                    (partial_slope * slope_std[i])**2
                )
                times_thr_std.append(time_thr_std)
    
    # Append results for the current threshold
    times.append(times_thr)
    times_std.append(times_thr_std)

print(times)
print(times_std)